In [ ]:
def periodDistance(distance,L = 35000):
    if distance > L/2 :
        return  distance - L
    else:
        if distance < -L/2 :
            return distance + L
        else:
            return distance


In [ ]:
def Transform_pos_vel(sat_Trees,cen_Tree,SnapRedshift,snapNum):
    import numpy as np
    # used to plot RMS_N -N
    
    
    #SnapRedshift = dict(np.load('/home/tnguser/data/SnapRedshift.npz'))['Redshift']
    #fields = ['SnapNum','SubhaloPos','SubhaloVel','Group_R_Crit200']
    
    h = 0.677
    a = 1 / (1+SnapRedshift[snapNum])
    f2 = a / h
    Num_sats = len(sat_Trees)
    
    index_cen = np.where(cen_Tree['SnapNum']==snapNum)[0]
    
    pos_cen = cen_Tree['SubhaloPos'][index_cen,:].reshape(3,)
    vel_cen = cen_Tree['SubhaloVel'][index_cen,:].reshape(3,)
    
    pos_sat = np.zeros([0,3])
    vel_sat = np.zeros([0,3])
    
        
    for i in np.arange(Num_sats):
        index_sat = np.where(sat_Trees[i]['SnapNum']==snapNum)[0]
        try:
            pos_sat = np.append(pos_sat,sat_Trees[i]['SubhaloPos'][index_sat,:].reshape(1,3),axis=0)
            vel_sat = np.append(vel_sat,sat_Trees[i]['SubhaloVel'][index_sat,:].reshape(1,3),axis=0)
        except ValueError:
            continue
            
    Num_sats = len(pos_sat[:,0])
            
    
    for i in np.arange(Num_sats):
        for j in np.arange(3):
            d = pos_sat[i,j]-pos_cen[j]                    # unit ckpc/h
            cDistance = periodDistance(distance=d)         # period box
            pos_sat[i,j] = cDistance * f2                  # unit kpc
            vel_sat[i,j] = vel_sat[i,j] - vel_cen[j]       # unit km/s
            
        
    return pos_sat, vel_sat
    

In [ ]:
def ratio_axis(pos):
    import numpy as np
    
    M = np.zeros((3,3))
    r_0 = np.zeros((1,3))
    
    r_0[0,0] = np.mean(pos[:,0])
    r_0[0,1] = np.mean(pos[:,1])
    r_0[0,2] = np.mean(pos[:,2])
    
    pos = pos - r_0
    
    for i in np.arange(pos.shape[0]):
        v = np.array([pos[i,:]])
        M += np.dot(v.T,v)
        
    eigval,eigvec = np.linalg.eig(M)
    index         = np.argsort(eigval)
    index_mindir  = index[0]
    
    return np.sqrt(eigval[index[0]]/eigval[index[2]]), np.sqrt(eigval[index[1]]/eigval[index[2]]),eigvec[:,index_mindir]

In [ ]:
def isotropic_pos(N):
    
    import numpy as np
    
    u = np.random.normal(0,1,N)
    v = np.random.normal(0,1,N)
    w = np.random.normal(0,1,N)
    
    norm = np.sqrt(u**2 + v**2 + w**2)
    
    x = u / norm
    y = v / norm
    z = w / norm
    
    return np.vstack((x,y,z)).T

In [ ]:
def coherent_iso_fast(sat_class_pos,sat_class_vel,N):
    import itertools
    
    L = sat_class_pos.shape[0]
    sat_class_ang     = np.zeros((L,3))
    #sat_class_ang_norm = np.zeros((L,3))
    for i in np.arange(L):
        sat_class_ang[i,:] = np.cross(sat_class_pos[i,:],sat_class_vel[i,:])
        sat_class_ang[i,:] = sat_class_ang[i,:] / np.linalg.norm(sat_class_ang[i,:])
    
    indexes = np.arange(L)
    index_comb  = np.array(list(itertools.combinations(indexes,N)))
    
    MEANS = isotropic_pos(1000)
    means_1 = np.zeros((0,3))
    RMSs_1  = np.zeros(0)
    means = np.zeros((0,3))
    RMSs  = np.zeros(0)
    
    for mean in MEANS:
        RMS  = np.abs(np.dot(sat_class_ang,mean))
        #RMS  = np.dot(sat_class_ang[com,:],mean)
        RMS  = np.mean(np.square(np.arccos(RMS)))
        RMS  = np.sqrt(RMS)

        means_1 = np.append(means_1,np.array([mean]),axis=0)
        RMSs_1  = np.append(RMSs_1,RMS)
        
    index = np.argsort(RMSs_1)[0]
    MEAN  = means_1[index]
    
    for i in np.arange(L):
        cos_theta = np.dot(sat_class_ang[i],MEAN)
        if cos_theta<0:
            sat_class_ang[i] = sat_class_ang[i] * (-1)
    
    for com in index_comb:
        mean = np.mean(sat_class_ang[com,:],axis=0)
        mean/= np.linalg.norm(mean)
#         RMS  = np.abs(np.dot(sat_class_ang[com,:],mean))
        RMS  = np.dot(sat_class_ang[com,:],mean)
        RMS  = np.mean(np.square(np.arccos(RMS)))
        RMS  = np.sqrt(RMS)
        
        means = np.append(means,np.array([mean]),axis=0)
        RMSs  = np.append(RMSs,RMS)
        
    index = np.argsort(RMSs)[0]
    
    return np.rad2deg(RMSs[index]),means[index],index_comb[index]
    
    

In [ ]:
def Draw(cen_find_ID,sat_find_IDs,snapNum,basePath,num):    
    import numpy as np 
    import illustris_python as il
    import matplotlib.pyplot as plt
    from matplotlib import animation

    SnapRedshift = dict(np.load('/home/tnguser/data/SnapRedshift.npz'))['Redshift']

    sat_Trees   = []
    fields_cen = ['SnapNum','SubhaloPos','SubhaloVel','SubfindID',
                  'SubhaloGrNr','Group_M_Crit200','Group_R_Crit200',
                  'SubhaloMassInRadType','GroupMass']
    
    fields_sat = ['SnapNum','SubhaloPos','SubhaloVel','SubfindID',
                  'SubhaloGrNr','SubhaloMassInRadType',
                  'SubhaloStellarPhotometrics']

    cen_Tree = il.sublink.loadTree(basePath,snapNum,id=cen_find_ID,fields=fields_cen,
                                           onlyMPB=True)
    for Id in sat_find_IDs:
        sat_Trees.append(il.sublink.loadTree(basePath,snapNum,id=Id,fields=fields_sat,
                                           onlyMPB=True))
    #print(num)
    sat_class_pos, sat_class_vel = Transform_pos_vel(sat_Trees,
                                                         cen_Tree,SnapRedshift,snapNum=num)
    
        
    
    return sat_class_pos, sat_class_vel, cen_Tree, sat_Trees

In [ ]:
def fun_14_iso_fast(haloID,SNums,basePath, snapNum,address,number):
    Data2 = dict(np.load(address))
    FoFGroup = il.groupcat.loadSingle(basePath, snapNum, haloID)
    cen_find_ID = FoFGroup['GroupFirstSub']
    index = np.where(Data2['GroupID']==haloID)
    #print(index)
    sat_find_IDs = Data2['Sat_IDs'][index]
    print(sat_find_IDs)
    sat_find_IDs = sat_find_IDs.reshape(sat_find_IDs.shape[1],)
    #print(sat_find_IDs)
    for num in SNums:
        pos,vel,c_tree,s_trees = Draw(cen_find_ID,sat_find_IDs,snapNum,basePath,num)
        ca_ratio, ba_ratio,eigvec = ratio_axis(pos)
        RMSs = np.zeros(0)
        MEANs= np.zeros((0,3))
        for j in number:
            RMS,mean,index = coherent_iso_fast(pos,vel,j)
            RMSs = np.append(RMSs,RMS)
            MEANs= np.append(MEANs,mean.reshape((1,3)),axis=0)
    return c_tree,s_trees, MEANs, RMSs, ca_ratio, ba_ratio, eigvec,pos,vel

In [ ]:
def AngMome(basePath,snapNum,ID):
    
    SnapRedshift = dict(np.load('/home/tnguser/data/SnapRedshift.npz'))['Redshift']
    a  = 1 / (1+SnapRedshift[snapNum])
    h  = 0.677
    f1 = 10**10 / h
    f2 = a / h
    
    fields = ['Coordinates','Velocities','GFM_StellarFormationTime','Masses']

    '''load position, and vercility of stars in the subhalo'''

    parData = il.snapshot.loadSubhalo(basePath,snapNum,ID,'star',fields=fields)

    '''find the the position and velocity of the subhalo'''

    subhaloInf = il.groupcat.loadSingle(basePath,snapNum,subhaloID=ID)
    subhaloPos = subhaloInf['SubhaloPos']
    subhaloVel = subhaloInf['SubhaloVel']
    starNum = subhaloInf['SubhaloLenType'][4]
    R_M = subhaloInf['SubhaloHalfmassRadType'][4] * f2     # kpc

    if starNum == 0:
        raise Exception('erro')
    
    '''reset the positions and verlosities '''

    for i in np.arange(starNum):
        for j in np.arange(3):
            d = parData['Coordinates'][i,j]-subhaloPos[j]  # unit ckpc/h
            cDistance = periodDistance(distance=d)         # period box
            parData['Coordinates'][i,j] = cDistance * f2    # unit kpc
            parData['Velocities'][i,j] = parData['Velocities'][i,j] * np.sqrt(a) - subhaloVel[j]  #km/s
            #parData['Velocities'][i,j] += H_z_0 * parData['Coordinates'][i,j] * 0.1  # change unit to km/s

    '''slect the real star particles and radius <3*R_M'''
    obj = []
    starRad = np.zeros(starNum)
    
    for i in np.arange(starNum): #get the radii of every star and wind particle
        starRad[i] = np.linalg.norm(parData['Coordinates'][i,:])

    parData['radius'] = starRad
    
    for i in np.arange(starNum):
        if parData['GFM_StellarFormationTime'][i] <= 0 or starRad[i] >= 6.3*R_M :   
            obj.append(i)

    fields = ['Coordinates','Velocities','GFM_StellarFormationTime','Masses','radius']
    for field in fields:
        parData[field] = np.delete(parData[field],obj,axis = 0)

    starNum = len(parData['GFM_StellarFormationTime']) # the number of real stars within 3*R_M
    if starNum == 0:
        raise Exception('erro')

    '''calculate the specific angular momentum'''

    angularMom = np.zeros([starNum,3])
    angularNorm = np.zeros(starNum)
    angularTotalMom = np.zeros(3)
    for i in np.arange(starNum):
        angularMom[i,:] = np.cross(parData['Coordinates'][i,:],parData['Velocities'][i,:])
        angularTotalMom = angularMom[i,:] + angularTotalMom

    angularTotalMom = angularTotalMom / np.linalg.norm(angularTotalMom)
    
    return angularTotalMom, parData['Coordinates'], parData['Masses']*f1

In [ ]:
import numpy as np
import illustris_python as il

basePath = '/home/tnguser/sims.TNG/TNG50-1/output/'
snapNum  = 99
# obs_RMS = dict(np.load('/home/tnguser/data/RMS_N_abs.npz'))['RMS_N']
# obs_RMS = dis_arr2d(obs_RMS,q=84)
Address = ['/home/tnguser/data/infall_time_14.npz'] 

address= Address[0]
number =  np.arange(11)+4
haloID = 395
SNums  = [99]

c_tree,s_trees, MEANs, RMSs, ca_ratio, ba_ratio,eigvec,pos,vel = fun_14_iso_fast(haloID,SNums,basePath, snapNum,address,number)


In [ ]:
basePath = '/home/tnguser/sims.TNG/TNG50-1/output/'
snapNum  = 99
ID = 589873
ang_mome, star_pos, star_mass = AngMome(basePath,snapNum,ID)

In [ ]:
def RMS_height(pos,N):
    import numpy as np
    
    Height = np.zeros(N)
    
    r_0 = np.mean(pos,axis=0)
    pos = pos - r_0
    
    Norm_n = isotropic_pos(N)
    
    for i in np.arange(Norm_n.shape[0]):
        arr = np.dot(pos,Norm_n[i])
        Height[i] = np.sqrt(np.mean(arr**2))
            
    index = np.argsort(Height)
    
    return Height[index[0]],Norm_n[index[0]],r_0
    
    

In [ ]:
def rotationMat(x,y,z,theta):
    w = np.array([[0,-z,y],[z,0,-x],[-y,x,0]])
    I = np.identity(3)
    R = I + w*np.sin(theta) + np.dot(w,w)*(1-np.cos(theta))

    return R

In [ ]:
def R_2(ang_mome,eigvec):
    z  = np.array([0,0,1])
    x  = np.array([1,0,0])
    ang_mome = ang_mome / np.linalg.norm(ang_mome)
    eigvec = eigvec / np.linalg.norm(eigvec)
    v1 = np.cross(ang_mome,z)
    theta1 = np.arccos(np.dot(ang_mome,z))
    v1 /= np.linalg.norm(v1)
    r1 = rotationMat(v1[0],v1[1],v1[2],theta1)
    #print(np.dot(r1,ang_mome))
    eigvec = np.dot(r1,eigvec)
    eigvec /= np.linalg.norm(eigvec)
    v2 = np.cross(eigvec,z)
    v2 /= np.linalg.norm(v2)
    theta2 = np.arccos(np.dot(v2,x))
    if v2[1] > 0:
        theta2 *= -1
    r2 = rotationMat(0,0,1,theta2+np.pi*(0))
    #r2 = rotationMat(0,0,1,theta2)
    #print(np.dot(r2,eigvec))
    
    return np.dot(r2,r1)

In [ ]:
def x1(x2,n2,n1,C):
    
    return (C-x2*n2) / n1

In [ ]:
import numpy as np

L=14
a=0
b=1
c=2
height = rms_h
Height = [0,height,-height]
offset = r_0
eigvec_norm = norm_n
x2 = np.linspace(-125,125,100)
x1s = np.zeros((3,100))

R_Matrix = R_2(ang_mome,eigvec_norm)
# R_Matrix = R_1(eigvec_norm)
POS = np.zeros((L,3))
VEL = np.zeros((L,3))
for i in np.arange(L):
    norm  = np.linalg.norm(pos[i])
    POS[i] = np.dot(R_Matrix,pos[i])
    POS[i] = POS[i] / np.linalg.norm(POS[i])
    #print(np.linalg.norm(POS[i]))
    POS[i] *= norm
    
    norm  = np.linalg.norm(vel[i])
    VEL[i] = np.dot(R_Matrix,vel[i])
    VEL[i] = VEL[i] / np.linalg.norm(VEL[i])
    #print(np.linalg.norm(VEL[i]))
    VEL[i] *= norm

eig_dir = np.dot(R_Matrix,eigvec_norm)
eig_dir /= np.linalg.norm(eig_dir)
offset = np.dot(R_Matrix,offset)

for i in np.arange(len(Height)):
    x1s[i] = x1(x2=x2,n2=eig_dir[2],n1=eig_dir[1],C=Height[i])
    
X2  = x2  + offset[2]
X1s = x1s + offset[1]



In [ ]:
np.savez('/home/tnguser/data/sat_plane.npz',VEL=VEL,POS=POS,X2=X2,X1s=X1s)

In [ ]:
import numpy as np

data = dict(np.load('/home/tnguser/data/sat_plane.npz'))

VEL = data['VEL']
POS = data['POS']
X2  = data['X2']
X1s = data['X1s']

### SP_plane plot

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.colors as colors

L=14
a=0
b=1
c=2

fig, ax = plt.subplots(figsize = (6,6))

for i in np.arange(L):
    if VEL[i,a] > 0:
        ax.scatter(POS[i,b],POS[i,c],marker='*',c='r',s=100)
    else:
        ax.scatter(POS[i,b],POS[i,c],marker='*',c='b',s=100)
        

ax.plot(X1s[0],X2,'-k')
ax.plot(X1s[1],X2,'--k')
ax.plot(X1s[2],X2,'--k')

ax.hlines(y=0,xmin= -10,xmax=10,colors='#000000',linewidth=4)
    
ax.axis('equal')
#ax.set_xlim(left=-40,right=40)
#ax.set_xticks(ticks=np.arange(-125,125,25))
ax.set_yticks(ticks=np.arange(-125,125,25))
ax.set_xlabel(r'y / kpc',fontsize=16)
ax.set_ylabel(r'z / kpc',fontsize=16)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
ax.spines['left'].set_linewidth(1.3)
ax.spines['bottom'].set_linewidth(1.3)
ax.spines['top'].set_linewidth(1.3)
ax.spines['right'].set_linewidth(1.3)
#plt.savefig('/home/tnguser/code/TheGreatPlane/Vedio_caratio/postion.pdf',bbox_inches='tight',dpi=250)

### $\theta_{k}-k$ plot 

In [ ]:
obs_RMS   = dict(np.load('/home/tnguser/data/RMS_N_abs_14_1000.npz'))
ANGM_N = obs_RMS['MEAN_ANGM_N']
ANGM_N.shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def theta(v1,v2):
    import numpy as np
    
    v1 = v1.reshape((3,))
    v2 = v2.reshape((3,))
    return np.rad2deg(np.arccos(np.abs(np.dot(v1,v2))))

L = 11
obs_ratio = dict(np.load('/home/tnguser/data/ratio_14_1000.npz'))
obs_RMS   = dict(np.load('/home/tnguser/data/RMS_N_iso_fast_14_1000.npz'))

Axis = obs_ratio['minor_axis']
Axis /= np.linalg.norm(Axis,axis=1).reshape(Axis.shape[0],1)
#print(np.linalg.norm(Axis,axis=1))

ANGM_N = obs_RMS['MEAN_ANGM_N']
for i in np.arange(len(ANGM_N)):
    arry = ANGM_N[i]
    ANGM_N[i] /= np.linalg.norm(arry,axis=1).reshape(arry.shape[0],1)
    #print(np.all(np.linalg.norm(ANGM_N[i],axis=1)==1))
    

obs_theta_N = np.zeros(ANGM_N.shape[:2])
sim_theta_N = np.zeros(L)

for i in np.arange(L):
    
    sim_theta_N[i] = theta(eigvec,MEANs[i,:])
    
for i in np.arange(ANGM_N.shape[0]):
    for j in np.arange(ANGM_N.shape[1]):
        obs_theta_N[i,j] = theta(Axis[j],ANGM_N[i,j,:])

obs_h = np.percentile(obs_theta_N,84,axis=1)
obs_m = np.percentile(obs_theta_N,50,axis=1)
obs_l = np.percentile(obs_theta_N,16,axis=1)

number = 4 + np.arange(L)

fig, ax = plt.subplots()
# ax.scatter(number,obs_h,label='MW_h')
# ax.scatter(number,obs_m,label='MW_m')
# ax.scatter(number,obs_l,label='MW_l')

uperr  = obs_h - obs_m
lowerr = obs_m - obs_l

yerr = np.vstack((lowerr,uperr))

ax.scatter(number,sim_theta_N,label='model galaxy',marker='^',c='m')
ax.errorbar(x=number,y=obs_m,yerr=yerr,fmt='oy', linewidth=2, capsize=6)

ax.set_xlabel('k',fontsize=16)
ax.set_ylabel(r'${\rm \theta_{k}(deg)}$',fontsize=16)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
ax.spines['left'].set_linewidth(1.3)
ax.spines['bottom'].set_linewidth(1.3)
ax.spines['top'].set_linewidth(1.3)
ax.spines['right'].set_linewidth(1.3)
ax.legend(fontsize=10)

#plt.savefig('/home/tnguser/code/TheGreatPlane/Vedio_caratio/theta_N.pdf',bbox_inches='tight',dpi=250)

### $\Delta_{k}-k$plot

In [ ]:
def dis_arr2d(arr,p=2,m=50,q=98):
    import numpy as np
    
    Len = len(arr[:,0])
    low  = np.percentile(arr,p,axis=1).reshape(1,Len)
    med  = np.percentile(arr,m,axis=1).reshape(1,Len)
    high = np.percentile(arr,q,axis=1).reshape(1,Len)
    result = np.concatenate((low,med,high),axis=0)
    return result

In [ ]:
import numpy as np
RMSs = dict(np.load('/home/tnguser/data/RMSs_14_iso_fast.npz'))['RMSs']

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import illustris_python as il


obs_RMS = dict(np.load('/home/tnguser/data/RMS_N_iso_fast_14_1000.npz'))['RMS_N']
obs_RMS = dis_arr2d(obs_RMS,p=16,q=84)

uperr  = obs_RMS[2,:] - obs_RMS[1,:]
lowerr = obs_RMS[1,:] - obs_RMS[0,:]
yerr = np.vstack((lowerr,uperr))

fig, ax = plt.subplots()

number =  np.arange(11)+4

ax.scatter(number,RMSs,label='model galaxy',marker='^',c='m')
ax.errorbar(x=number,y=obs_RMS[1,:],yerr=yerr,fmt='oy', linewidth=2, capsize=6)

ax.set_xlabel('k',fontsize=16)
ax.set_ylabel(r'${\rm \Delta_{k}(deg)}$',fontsize=16)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
ax.spines['left'].set_linewidth(1.3)
ax.spines['bottom'].set_linewidth(1.3)
ax.spines['top'].set_linewidth(1.3)
ax.spines['right'].set_linewidth(1.3)
ax.legend(fontsize=10)

#plt.savefig('/home/tnguser/code/TheGreatPlane/Vedio_caratio/RMS_N_abs.pdf',bbox_inches='tight',dpi=250)

plt.show()


### $\Delta_{k}-k$plot without abs

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import illustris_python as il


obs_RMS = dict(np.load('/home/tnguser/data/RMS_N_no_abs_14_1000.npz'))['RMS_N']
obs_RMS = dis_arr2d(obs_RMS,p=16,q=84)

uperr  = obs_RMS[2,:] - obs_RMS[1,:]
lowerr = obs_RMS[1,:] - obs_RMS[0,:]
yerr = np.vstack((lowerr,uperr))

fig, ax = plt.subplots()

number =  np.arange(11)+4

ax.scatter(number,RMSs,label='model galaxy',marker='^',c='m')
ax.errorbar(x=number,y=obs_RMS[1,:],yerr=yerr,fmt='oy', linewidth=2, capsize=6)

ax.set_xlabel('k',fontsize=16)
ax.set_ylabel(r'${\rm \Delta_{k}(deg)}$',fontsize=16)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
ax.spines['left'].set_linewidth(1.3)
ax.spines['bottom'].set_linewidth(1.3)
ax.spines['top'].set_linewidth(1.3)
ax.spines['right'].set_linewidth(1.3)
ax.legend(fontsize=10)

#plt.savefig('/home/tnguser/code/TheGreatPlane/Vedio_caratio/RMS_N_no_abs.pdf',bbox_inches='tight',dpi=250)

plt.show()


### evolution plot

In [ ]:
def AgeRedshift(x):
    from scipy import integrate
    h =  0.6774
    Omega_L0 = 0.6911
    Omega_r0 = 0       #4.2 * 10**(-5) / h**2
    Omega_m0 = 0.3089

    def E(z):
        return np.sqrt(Omega_L0 - Omega_r0*(1+z)**2 + Omega_m0 * (1+z)**3 + Omega_r0*(1+z)**4)
        #return np.sqrt(Omega_L0 + Omega_m0 * (1+z)**3 + Omega_r0*(1+z)**4)
    def f(z):
        return 9.785 / (h*E(z)*(1+z))


    return integrate.quad(lambda z:f(z),x,np.inf)[0]

In [ ]:
import numpy as np
SnapRedshift = dict(np.load('/home/tnguser/data/SnapRedshift.npz'))['Redshift']
SNAP = np.arange(100)
Age  = np.zeros(0)
for snap in SNAP:
    Age = np.append(Age,AgeRedshift(SnapRedshift[snap]))

In [ ]:
def Transform_pos_vel(sat_Trees,cen_Tree,snapNum):
    import numpy as np
    
    SnapRedshift = dict(np.load('/home/tnguser/data/SnapRedshift.npz'))['Redshift']
    #fields = ['SnapNum','SubhaloPos','SubhaloVel','Group_R_Crit200']
    
    h = 0.677
    a = 1 / (1+SnapRedshift[snapNum])
#     f2 = a / h
    f2 = 1 / h
    f3 = 1 / 10**3
    
    Num_sats = len(sat_Trees)
    
    index_cen = np.where(cen_Tree['SnapNum']==snapNum)[0]
    
    pos_cen = cen_Tree['SubhaloPos'][index_cen,:].reshape(3,)
    vel_cen = cen_Tree['SubhaloVel'][index_cen,:].reshape(3,)
    
    pos_sat = np.zeros([0,3])
    vel_sat = np.zeros([0,3])
    #mass_sat = np.zeros(0)
    
    for i in np.arange(Num_sats):
        index_sat = np.where(sat_Trees[i]['SnapNum']==snapNum)[0]
        try:
            pos_sat = np.append(pos_sat,sat_Trees[i]['SubhaloPos'][index_sat,:].reshape(1,3),axis=0)
            vel_sat = np.append(vel_sat,sat_Trees[i]['SubhaloVel'][index_sat,:].reshape(1,3),axis=0)
        except ValueError:
            miss_index = i
            #print('missing sat index:', i)
            continue
            
    Num_sats = len(pos_sat[:,0])
        
    
    for i in np.arange(Num_sats):
        for j in np.arange(3):
            d = pos_sat[i,j]-pos_cen[j]                    # unit ckpc/h
            cDistance = periodDistance(distance=d)         # period box
            pos_sat[i,j] = cDistance * f2                  # unit ckpc
            vel_sat[i,j] = vel_sat[i,j] - vel_cen[j]       # unit km/s
            #vel_sat[i,j] = vel_sat[i,j] + H(SnapRedshift[snapNum])*pos_sat[i,j]*f3
            vel_sat[i,j] /=a                               #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! comoving
            
    
    if 'miss_index' in dir():
        return pos_sat,vel_sat,miss_index
    else:
        return pos_sat,vel_sat,np.inf
    

In [ ]:
def tree(cen_find_ID,sat_class_ID):
    
    import numpy as np 
    import illustris_python as il

    snapNum = 99
    SnapRedshift = dict(np.load('/home/tnguser/data/SnapRedshift.npz'))['Redshift']

    sat_Trees     = []
    sat_sec_Trees = []

    fields_cen = ['SnapNum','SubhaloPos','SubhaloVel','SubhaloGrNr']
    fields_sat = ['SnapNum','SubhaloPos','SubhaloVel','SubhaloGrNr']


    cen_Tree = il.sublink.loadTree(basePath,snapNum,id=cen_find_ID,fields=fields_cen,
                                           onlyMPB=True)
    for Id in sat_class_ID:
        sat_Trees.append(il.sublink.loadTree(basePath,snapNum,id=Id,fields=fields_sat,
                                           onlyMPB=True))

    return cen_Tree,sat_Trees

In [ ]:
def Thetas(sat_class_pos,sat_class_vel,minor_axis):
    
    N = sat_class_pos.shape[0]
    sat_class_ang     = np.zeros((N,3))
    #sat_class_ang_norm = np.zeros((N,3))
    for i in np.arange(N):
        sat_class_ang[i,:] = np.cross(sat_class_pos[i,:],sat_class_vel[i,:])
        sat_class_ang[i,:] = sat_class_ang[i,:] / np.linalg.norm(sat_class_ang[i,:])

    cos_thetas = np.dot(sat_class_ang,minor_axis) 
    #cos_thetas = np.abs(cos_thetas)
    thetas = np.rad2deg(np.arccos(cos_thetas))
    #Thetas = np.sort(thetas)
    
    return thetas.reshape(1,N)   

In [ ]:
def cluster(pos):
    
    N = pos.shape[0]
    arr = np.zeros(N)
    
    pos = pos - np.mean(pos,axis=0)
    
    for i in np.arange(N):
        dis = pos - pos[i]
        norm = np.linalg.norm(dis,axis=1)
        print(pos[i].shape)
        arr[i] = np.sum(norm) 
        
    return arr
    

In [ ]:
def fun_dis(sat_ID_cluster,ax,haloID,basePath, snapNum,time):
    
#     Data = dict(np.load(address))
#     index = np.where(Data['GroupID']==haloID)
#     sat_class_ID = Data['Sat_IDs'][index]

    sat_class_ID = sat_ID_cluster
    print(len(sat_class_ID))
    N = len(sat_class_ID)
    
    FoFGroup = il.groupcat.loadSingle(basePath, snapNum, haloID)
    cen_find_ID = FoFGroup['GroupFirstSub']
    
    minor_axis = np.array([0,1,0])
    
    #Snaps  = np.arange(100-snap) + snap
    Snaps  = np.arange(0,100)
    
    Thes = np.zeros(0)
    Snaps_update = np.zeros(0)
    
    for Snap in Snaps:
        cen_Tree,sat_Trees = tree(cen_find_ID,sat_class_ID)
        pos,vel,miss_index = Transform_pos_vel(sat_Trees,cen_Tree,snapNum=Snap)
        #print(len(pos))
        if len(pos) != N:
            continue
            
        theta,mean,index   = coherent_iso_fast(pos,vel,N)

#         theta,mean,index   = coherent(pos,vel,N)
        Thes    = np.append(Thes,theta)
        Snaps_update = np.append(Snaps_update,Snap)
    
    ages = np.zeros(0)
    for snap in Snaps_update:
        index = np.where(SNAP==snap)[0]
        ages = np.append(ages,Age[index])
    ages = 13.8 - ages
    
    Index = np.in1d(Snaps_update,time)
    
    ax.plot(ages,Thes)
    
    print('theta:',Thes[Index])
    
    ax.axvline(x=ages[Index][0],linestyle='dotted',color='black')
    ax.axvline(x=ages[Index][1],linestyle='dotted',color='darkgreen')
    ax.axvline(x=ages[Index][2],linestyle='dotted',color='brown')
    ax.axvline(x=ages[Index][3],linestyle='dotted',color='indigo')

        
    
    
    print('Age:',ages[Index])

#     ax.set_title('model galaxy',fontsize=16)
    ax.set_ylabel(r'$ \sigma_{\theta}\ /\ (deg)$',fontsize=16)
    ax.set_xlabel('Lookback Time/(Gyr)',fontsize=16)
    
    #ax.set_yticklabels(fontdict={'fontsize': 16})
    ax.tick_params(labelsize=16)
    ax.spines['left'].set_linewidth(1.3)
    ax.spines['bottom'].set_linewidth(1.3)
    ax.spines['top'].set_linewidth(1.3)
    ax.spines['right'].set_linewidth(1.3)
    #ax.legend(fontsize=10)
    return

In [ ]:
def fun_theta(sat_ID_cluster,ax,haloID,basePath, snapNum,time):
    import numpy as np
    import illustris_python as il
    import matplotlib.pyplot as plt
    
    SnapRedshift = dict(np.load('/home/tnguser/data/SnapRedshift.npz'))['Redshift']
    sat_class_ID = sat_ID_cluster
    print(len(sat_class_ID))
    N = len(sat_class_ID)
    
    FoFGroup = il.groupcat.loadSingle(basePath, snapNum, haloID)
    cen_find_ID = FoFGroup['GroupFirstSub']
    
    minor_axis = np.array([1,0,0])
    
    #Snaps  = np.arange(100-snap) + snap
    Snaps  = np.arange(0,100)
    
    Thes = np.zeros([0,N])
    Snaps_update = np.zeros(0,dtype=int)
    
    for Snap in Snaps:
        cen_Tree,sat_Trees = tree(cen_find_ID,sat_class_ID)
        pos,vel,miss_index = Transform_pos_vel(sat_Trees,cen_Tree,snapNum=Snap)
        #print(len(pos))
        if len(pos) != N:
            continue
        theta   = Thetas(pos,vel,minor_axis)
        Thes    = np.append(Thes,theta,axis=0)
        Snaps_update = np.append(Snaps_update,Snap)
    
    ages = np.zeros(0)
    for snap in Snaps_update:
        index = np.where(SNAP==snap)[0]
        ages = np.append(ages,Age[index])
    ages = 13.8 - ages
    
    Index = np.in1d(Snaps_update,time)
#     Redshifts = SnapRedshift[Snaps_update]
    
    
    for i in np.arange(N):
        ax.plot(ages,Thes[:,i])
        
    ax.axvline(x=ages[Index][0],linestyle='dotted',color='black')
    ax.axvline(x=ages[Index][1],linestyle='dotted',color='darkgreen')
    ax.axvline(x=ages[Index][2],linestyle='dotted',color='brown')
    ax.axvline(x=ages[Index][3],linestyle='dotted',color='indigo')

    ax.set_ylabel(r'${\rm\theta\  / \  (deg)}$',fontsize=16,labelpad=0.006)
    #ax.set_xlabel('Lookback Time / (Gyr)',fontsize=16)
    
    #ax.set_yticklabels(fontdict={'fontsize': 16})
    ax.tick_params(labelsize=16,pad=0.01)
    ax.spines['left'].set_linewidth(1.3)
    ax.spines['bottom'].set_linewidth(1.3)
    ax.spines['top'].set_linewidth(1.3)
    ax.spines['right'].set_linewidth(1.3)
    #ax.legend(fontsize=10)
    return
        

    

In [ ]:
def fun_ratio(sat_ID_cluster,ax,haloID,basePath, snapNum,time):
    
    import numpy as np
    import illustris_python as il
    import matplotlib.pyplot as plt
    
    SnapRedshift = dict(np.load('/home/tnguser/data/SnapRedshift.npz'))['Redshift']
    sat_class_ID = sat_ID_cluster
    print(len(sat_class_ID))
    N = len(sat_class_ID)
    
    FoFGroup = il.groupcat.loadSingle(basePath, snapNum, haloID)
    cen_find_ID = FoFGroup['GroupFirstSub']
    
    #Snaps  = np.arange(100-snap) + snap
    Snaps  = np.arange(0,100)
    
    Thes = np.zeros(0)
    Snaps_update = np.zeros(0,dtype=int)
    
    for Snap in Snaps:
        cen_Tree,sat_Trees = tree(cen_find_ID,sat_class_ID)
        pos,vel,miss_index = Transform_pos_vel(sat_Trees,cen_Tree,snapNum=Snap)
        #print('miss_index',miss_index)
        if len(pos) != N:
            print('missing snap:',Snap,'left num:',len(pos))
            continue
        ca_ratio, ba_ratio,eigvec = ratio_axis(pos)
        Thes = np.append(Thes,ca_ratio)
        Snaps_update = np.append(Snaps_update,Snap)
    
    ages = np.zeros(0)
    for snap in Snaps_update:
        index = np.where(SNAP==snap)[0]
        ages = np.append(ages,Age[index])
    ages = 13.8 - ages
    
    Index = np.in1d(Snaps_update,time)
#     Redshifts = SnapRedshift[Snaps_update]
    
    ax.plot(ages,Thes)
#     ax.scatter(Redshifts[Index][0],Thes[Index][0],label='first infall')
#     ax.scatter(Redshifts[Index][1],Thes[Index][1],label='last  infall')
    
    ax.axvline(x=ages[Index][0],linestyle='dotted',color='black',label='1st sat infall')
    ax.axvline(x=ages[Index][1],linestyle='dotted',color='darkgreen',label='5th sat infall')
    ax.axvline(x=ages[Index][2],linestyle='dotted',color='brown',label='10th sat infall')
    ax.axvline(x=ages[Index][3],linestyle='dotted',color='indigo',label='11th sat infall')
    
    #ax.axhline(y=Thes[Index][0],linestyle='dotted',color='darkorange')
    
    

    ax.set_ylabel(r'c/a',fontsize=16)
    ax.set_xlabel('Lookback Time / (Gyr)',fontsize=16)
    ax.tick_params(labelsize=16)
    
    ax.spines['left'].set_linewidth(1.3)
    ax.spines['bottom'].set_linewidth(1.3)
    ax.spines['top'].set_linewidth(1.3)
    ax.spines['right'].set_linewidth(1.3)
    ax.legend(fontsize=8)
    return
        

In [ ]:
import numpy as np
import illustris_python as il
import matplotlib.pyplot as plt

def fun(address,haloID,basePath, snapNum,Snap):
    Data = dict(np.load(address))
    index = np.where(Data['GroupID']==haloID)
    sat_class_ID = Data['Sat_IDs'][index]
    
    N = len(sat_class_ID[0])
    sat_class_ID = sat_class_ID.reshape(N,)
    FoFGroup = il.groupcat.loadSingle(basePath, snapNum, haloID)
    cen_find_ID = FoFGroup['GroupFirstSub']
    
#     #Snaps  = np.arange(100-snap) + snap
#     Snaps  = np.arange(30,100)
    
    cen_Tree,sat_Trees = tree(cen_find_ID,sat_class_ID)
    pos,vel,miss_index = Transform_pos_vel(sat_Trees,cen_Tree,snapNum=Snap)
    print(len(pos))
    dis = cluster(pos)
    
    index = np.argsort(dis)[:-2]
    sat_class_ID = np.delete(sat_class_ID,miss_index)
    sat_ID_cluster = sat_class_ID[index]
    
    return sat_ID_cluster

        
basePath = '/home/tnguser/sims.TNG/TNG50-1/output/'
snapNum  = 99
HaloID = [395]
Address= ['/home/tnguser/data/infall_time_14.npz']
Snap = 14
address = Address[0]
haloID = HaloID[0]
sat_ID_cluster = fun(address,haloID,basePath, snapNum,Snap)

In [ ]:
import numpy as np
import illustris_python as il
import matplotlib.pyplot as plt

basePath = '/home/tnguser/sims.TNG/TNG50-1/output/'
snapNum  = 99
haloID = 395
time = np.array([28,33,86,99])

fig, axs = plt.subplots(3, 1, figsize=(9,6),sharex=True)
fig.subplots_adjust(hspace=0)

fun_dis(sat_ID_cluster,axs[0],haloID,basePath, snapNum,time)
fun_theta(sat_ID_cluster,axs[1],haloID,basePath, snapNum,time)
fun_ratio(sat_ID_cluster,axs[2],haloID,basePath, snapNum,time)
#plt.savefig('/home/tnguser/code/TheGreatPlane/Vedio_caratio/caRatio_and_theta_cluster_infall.pdf',bbox_inches='tight',dpi=250)